In [1]:
import os 
import numpy as np
from PIL import Image
import cv2
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import keras

Download the GHTSRB dataset at this address : https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

In [ ]:
#GHTSRB dataset

#assigning path for dataset
train_path='./GHTSRB/Train/'
test_path='./GHTSRB/Test/'

# Resizing the images to 30x30x3
IMG_HEIGHT = 30
IMG_WIDTH = 30
channels = 3

#finding total classes
NUM_CATEGORIES = len(os.listdir(train_path))


classes={0:'Speed limit(20km/hr)',
         1:'Speed limit(30km/hr)',
         2:'Speed limit(50km/hr)',
         3:'Speed limit(60km/hr)',
         4:'Speed limit(70km/hr)',
         5:'Speed limit(80km/hr)',
         6:'End of speed limt(80km/hr)',
         7:'Speed limit(100km/hr)',
         8:'Speed limit(120km/hr)',
         9:'No passing',
         10:'No passing vehicles over 3.5 tons',
         11:'Right- of- way at inetersection',
         12:'Priority road',
         13:'Yield',
         14:'Stop',
         15:'No vehicles',
         16:'Vehicles greater than 3.5 tons prohibited',
         17:'No entry',
         18:'General Caution',
         19:'Dangers curve left',
         20:'Dangers curve Right',
         21:'Double curve',
         22:'Bumpy road',
         23:'Slippery road',
         24:'Road narrows on the right',
         25:'Road work',
         26:'Traffic signals',
         27:'Pedestrians',
         28:'Children crossing ',
         29:'Bicycle crossing',
         30:'Beware of ice/snow',
         31:'Wild animal crossing',
         32:'End speed + passing limits',
         33:'Turn right ahead',
         34:' Turn left ahead',
         35:'Ahead only',
         36:'Go straight or right',
         37:'Go straight or left',
         38:'Keep right',
         39:'Kepp left',
         40:'Rounabout mandatory',
         41:'End of no passing',
         42:'End of no passingVehicle greater than 3.5tons'
        }


folders = os.listdir(train_path)

train_number = []
class_num = []

for folder in folders:
    train_files = os.listdir(train_path + '/' + folder)
    train_number.append(len(train_files))
    class_num.append(classes[int(folder)])
    
# Sorting the dataset on the basis of number of images in each class
zipped_lists = zip(train_number, class_num)
sorted_pairs = sorted(zipped_lists)
tuples = zip(*sorted_pairs)
train_number, class_num = [ list(tuple) for tuple in  tuples]

# Collecting Training data
image_data = []
image_labels = []

for i in range(NUM_CATEGORIES):
    path = train_path + str(i)
    images = os.listdir(path)

    for img in images:
        try:
            image = cv2.imread(path + '/' + img)
            image_fromarray = Image.fromarray(image, 'RGB')
            resize_image = image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))
            image_data.append(np.array(resize_image))
            image_labels.append(i)
        except:
            print("Error in " + img)


In [ ]:
# Changing the list to numpy array
image_data = np.array(image_data)
image_labels = np.array(image_labels)


#Shuffle the training data
shuffle_indexes = np.arange(image_data.shape[0])
np.random.shuffle(shuffle_indexes)
image_data = image_data[shuffle_indexes]
image_labels = image_labels[shuffle_indexes]

#split train and test data
X_train, X_val, y_train, y_val = train_test_split(image_data, image_labels, test_size=0.3, random_state=42, shuffle=True)

X_train = X_train/255 
X_val = X_val/255

print("X_train shape", X_train.shape)
print("X_valid shape", X_val.shape)

#one hot encoding of target
NUM_CATEGORIES=43
y_train = keras.utils.to_categorical(y_train, NUM_CATEGORIES)
y_val = keras.utils.to_categorical(y_val, NUM_CATEGORIES)

print("y_train shape", y_train.shape)
print("y_valid shape", y_val.shape)


In [ ]:
#Making model
model = keras.models.Sequential([    
    keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(IMG_HEIGHT,IMG_WIDTH,channels)),
    keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.BatchNormalization(axis=-1),
    
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.BatchNormalization(axis=-1),
    
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate=0.5),
    
    keras.layers.Dense(43, activation='softmax')
])

In [ ]:
#Compile the model
lr = 0.001
epochs = 30
opt = Adam(learning_rate=lr, weight_decay=lr / (epochs * 0.5))
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])



In [ ]:
#Augumenting the data and training
aug = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode="nearest")

In [ ]:
# Train model
history = model.fit(aug.flow(X_train, y_train, batch_size=32), epochs=epochs, validation_data=(X_val, y_val))

In [2]:
from keras.models import load_model
model=load_model("model.h5")

In [5]:
# Save model
model.save("model.h5")